<a href="https://colab.research.google.com/github/hochthom/OG-AI4Artists-2021/blob/main/Lecture4_StyleGAN_Example/StyleGAN_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from tqdm.notebook import tqdm
from stylegan2 import StyleGAN

In [2]:
!ls -l /content/drive/MyDrive/AI4Artists/data

total 383821
-rw------- 1 root root 393032041 Apr 29 09:07 images.zip


Here we need to define the path to our images. We can either link to a folder containing the inages or to a zip file.

In [3]:
IMAGE_SOURCE = '/content/drive/MyDrive/AI4Artists/data/images.zip'
print(os.path.exists(IMAGE_SOURCE))

True


Initialisation
==============

Let's build the StyleGAN model and initialize our data set.

In [4]:
model = StyleGAN(lr=0.0001, silent=False)
model.create_data(IMAGE_SOURCE)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           input_2[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 1024)         2048        lambda[0][0]                     
__________________________________________________________________________________________________
reshape (Reshape)               (None, 4, 4, 64)     0           dense_5[0][0]                    
____________________________________________________________________________________________

By loading a previously saved model we can continue training from that snapshot. 

In [5]:
model.load(0)

Training
========

Next we train the styleGAN for some epochs. The number of epochs depends on the dataset, but in general the longer the better (e.g. epochs ~ 40k).

In [5]:
epochs = 2001
for e in tqdm(range(epochs)):
    model.train()

Round    100: D =  6.2939, G = -4.0781, PL =  0.0511
Round    200: D =  5.3196, G =  1.1218, PL =  0.0227
Round    300: D =  1.7066, G =  1.4547, PL =  0.0118
Round    400: D =  0.8587, G =  0.6678, PL =  0.0083
Round    500: D =  1.7598, G = -0.2385, PL =  0.0061
Round    600: D =  1.2669, G =  0.2550, PL =  0.0056
Round    700: D =  1.1747, G =  5.1973, PL =  0.0042
Round    800: D =  0.6387, G =  3.4626, PL =  0.0039
Round    900: D =  1.5706, G =  2.7371, PL =  0.0034
Round   1000: D =  0.3038, G =  3.1711, PL =  0.0039
Saving model as m0 ...
Round   1100: D =  0.5595, G =  1.9705, PL =  0.0039
Round   1200: D =  1.0429, G =  0.4580, PL =  0.0060
Round   1300: D =  0.2206, G =  1.8080, PL =  0.0045
Round   1400: D =  0.7657, G =  2.5504, PL =  0.0042
Round   1500: D =  0.2241, G =  1.9436, PL =  0.0035
Round   1600: D =  0.4266, G =  2.5324, PL =  0.0041
Round   1700: D =  0.2113, G =  1.5934, PL =  0.0039
Round   1800: D =  0.2235, G =  3.2408, PL =  0.0034
Round   1900: D =  0.20